In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "wireshark", 1

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'epan/ftypes/ftype-integer.c'}
bug.buggy_lines={('epan/ftypes/ftype-integer.c', 422), ('epan/ftypes/ftype-integer.c', 423), ('epan/ftypes/ftype-integer.c', 429), ('epan/ftypes/ftype-integer.c', 428)}
bug.coverage_df=


36  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   615   
                                                                                  177   615   
                                                                                  199   615   
                                                                                  203   615   
                                                                                  205   615   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  615   
                                                                                  1713  615   
                                                                                  1715  615   
                                                                                  1716  615   
                                                                                  1717  615   

                                                                                         27  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   477   
                                                                                  177   477   
                                                                                  199   477   
                                                                                  203   477   
                                                                                  205   477   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  477   
                                                                                  1713  477   
                                                                                  1715  477   
                                                                                  1716  477   
                                                                                  1717  477   

                                                                                          9  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   281   
                                                                                  177   281   
                                                                                  199   281   
                                                                                  203   281   
                                                                                  205   281   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  281   
                                                                                  1713  281   
                                                                                  1715  281   
                                                                                  1716  281   
                                                                                  1717  281   

                                                                                          5  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   222   
                                                                                  177   222   
 

bug.failing_tests={'31', '3', '22'}
bug.failing_tests_GT={'31', '3', '22'}
bug.functions={('/home/workspace/epan/ftypes/ftypes.c', 'fvalue_ge'), ('/home/workspace/epan/dissectors/packet-lisp-tcp.c', 'proto_register_lisp_tcp'), ('/home/workspace/plugins/epan/wimax/msg_reg_req.c', 'proto_register_mac_mgmt_msg_reg_req'), ('/home/workspace/epan/dissectors/packet-zbee-zcl.c', 'dissect_zcl_attr_data_type_val'), ('/home/workspace/epan/prefs.c', 'prefs_set_preference_effect_fields'), , ...}
bug.id='wireshark-1'
bug.num_test_cases=36
bug.passing_tests={'35', '33', '21', '19', '8', '18', '28', '25', '17', '14', '36', '23', '5', '27', '2', '11', '10', '34', '7', '16', '6', '32', '20', '15', '24', '30', '1', '29', '13', '26', '9', '4', '12'}
bug.path_to_cov_df='/home/coinse/Workspace/defects4cpp/coverage/wireshark_1_buggy/coverage_df.pkl'
bug.path_to_patch='/home/coinse/Workspace/defects4cpp/defects4cpp/taxonomy/wireshark/patch/0001-buggy.patch'
bug.path_to_test_results='/home/coinse/Workspace/def

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                                  function                    line                                                                                              
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 174)   
                                                                                  177   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 177)   
                                                                                  199   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 199)   
                                                                                  203   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 203)   
                                                                                  205   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 205)   
...                                                                                                                                                                           ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1712)               
                                                                                  1713  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1713)               
                                                                                  1715  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1715)               
                                                                                  1716  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1716)               
                                                                                  1717  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1717)               

                                                                                        e_p  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   33    
                                                                                  177   33    
                                                                                  199   33    
                                                                                  203   33    
                                                                                  205   33    
...                                                                                     ..    
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  33    
                                                                                  1713  33    
                                                                                  1715  33    
                                                                                  1716  33    
                                                                                  1717  33    

                                                                                        e_f  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   3     
                                                                                  177   3     
                                                                                  199   3     
                                                                                  203   3     
             

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/epan/ftypes/ftype-integer.c', 'uinteger_to_repr')
fault idx: ('/home/workspace/epan/ftypes/ftype-integer.c', 'uinteger_to_repr', 429)
fault idx: ('/home/workspace/epan/ftypes/ftype-integer.c', 'uinteger_to_repr', 423)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr, 429)",33,3,0,0,2.029412,0.288675,81573.0,87686.0
1,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr, 423)",33,3,0,0,2.029412,0.288675,81573.0,87686.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2012)",29,3,4,0,2.147059,0.306186,18.0,19.0
1,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2002)",29,3,4,0,2.147059,0.306186,18.0,19.0
2,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2017)",29,3,4,0,2.147059,0.306186,18.0,19.0
3,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2015)",29,3,4,0,2.147059,0.306186,18.0,19.0
4,"(/home/workspace/epan/proto.c, save_same_name_hfinfo, 420)",29,3,4,0,2.147059,0.306186,18.0,19.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/prefs.c, register_preference, 1041)",2,1,31,2,0.941176,0.333333,89904.0,1.0
1,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2004)",29,3,4,0,2.147059,0.306186,18.0,19.0
2,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2000)",29,3,4,0,2.147059,0.306186,18.0,19.0
3,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2002)",29,3,4,0,2.147059,0.306186,18.0,19.0
4,"(/home/workspace/epan/packet.c, dissector_compare_filter_name, 2003)",29,3,4,0,2.147059,0.306186,18.0,19.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,174,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
1,177,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
2,199,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
3,203,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
4,205,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
...,...,...,...,...,...,...,...,...,...,...,...,...
103431,1712,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
103432,1713,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
103433,1715,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
103434,1716,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",436,33,3,0,0,2.029412,6707.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/epan/proto.c, save_same_name_hfinfo)",423,29,3,4,0,2.147059,3.0
1,"(/home/workspace/epan/packet.c, dissector_compare_filter_name)",2018,29,3,4,0,2.147059,3.0
2,"(/home/workspace/epan/addr_resolv.c, fgetline)",610,29,3,4,0,2.147059,3.0
3,"(/usr/include/x86_64-linux-gnu/sys/stat.h, nan)",469,31,3,2,0,2.088235,1061.0
4,"(/home/workspace/wsutil/wsgcrypt.c, ws_cmac_buffer)",50,31,3,2,0,2.088235,1061.0
5,"(/home/workspace/wsutil/wsgcrypt.c, hkdf_expand)",208,31,3,2,0,2.088235,1061.0
6,"(/home/workspace/wsutil/strtoi.c, ws_strtou64)",198,31,3,2,0,2.088235,1061.0
7,"(/home/workspace/wsutil/nstime.c, nstime_to_sec)",227,31,3,2,0,2.088235,1061.0
8,"(/home/workspace/wsutil/nstime.c, nstime_to_msec)",171,31,3,2,0,2.088235,1061.0
9,"(/home/workspace/wsutil/nstime.c, nstime_delta)",104,31,3,2,0,2.088235,1061.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",436,33,3,0,0,0.288675,7159.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/epan/prefs.c, register_preference)",1041,2,1,31,2,0.333333,1.0
1,"(/home/workspace/epan/proto.c, save_same_name_hfinfo)",423,29,3,4,0,0.306186,4.0
2,"(/home/workspace/epan/packet.c, dissector_compare_filter_name)",2018,29,3,4,0,0.306186,4.0
3,"(/home/workspace/epan/addr_resolv.c, fgetline)",610,29,3,4,0,0.306186,4.0
4,"(/home/workspace/wsutil/json_dumper.c, json_puts_string)",50,13,2,20,1,0.298142,64.0
5,"(/home/workspace/wsutil/json_dumper.c, json_dumper_end_array)",298,13,2,20,1,0.298142,64.0
6,"(/home/workspace/wsutil/json_dumper.c, json_dumper_begin_array)",286,13,2,20,1,0.298142,64.0
7,"(/home/workspace/writecap/pcapio.c, libpcap_write_packet)",231,13,2,20,1,0.298142,64.0
8,"(/home/workspace/writecap/pcapio.c, libpcap_write_file_header)",210,13,2,20,1,0.298142,64.0
9,"(/home/workspace/tshark.c, write_preamble)",3931,13,2,20,1,0.298142,64.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.058824,3689.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,436,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
1,434,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
2,430,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
3,429,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
4,427,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
5,426,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
6,423,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353
7,421,33,3,0,0,2.029412,0.288675,81573.0,87686.0,/home/workspace/epan/ftypes/ftype-integer.c,uinteger_to_repr,"(/home/workspace/epan/ftypes/ftype-integer.c, uinteger_to_repr)",4.0,0.507353


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,3851,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
1,3848,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
2,3810,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
3,3805,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
4,3794,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2775,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
229,2774,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
230,2773,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261
231,2772,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c,dissect_radiotap,"(/home/workspace/epan/dissectors/packet-ieee80211-radiotap.c, dissect_radiotap)",4.0,0.074261


Top 5:


,meth,vote,rank
0,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",17.113476,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,3424,31,3,2,0,2.088235,0.297044,17195.0,17904.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",4.0,0.074261
1,4917,32,3,1,0,2.058824,0.292770,25088.0,25797.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",5.0,0.058554
2,4915,32,3,1,0,2.058824,0.292770,25088.0,25797.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",5.0,0.058554
3,4912,32,3,1,0,2.058824,0.292770,25088.0,25797.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",5.0,0.058554
4,4911,32,3,1,0,2.058824,0.292770,25088.0,25797.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",5.0,0.058554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,3565,1,0,32,3,-0.029412,0.000000,97102.0,103436.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",33.0,0.000000
320,3563,1,0,32,3,-0.029412,0.000000,97102.0,103436.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",33.0,0.000000
321,3556,1,0,32,3,-0.029412,0.000000,97102.0,103436.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",33.0,0.000000
322,3553,1,0,32,3,-0.029412,0.000000,97102.0,103436.0,/home/workspace/epan/dissectors/packet-sip.c,dissect_sip_common,"(/home/workspace/epan/dissectors/packet-sip.c, dissect_sip_common)",33.0,0.000000


# coverage hierarchy

In [24]:
pd.set_option("display.max_columns", None)
print("[fault_idx covs]")
fault_covmat_idxs = []
for fault_meth, fault_idxs_of_meth in fault_meth2idx.items():
    fault_covmat_idxs += fault_idxs_of_meth
display(covmat.astype(bool).astype(int).loc[fault_covmat_idxs])

print("[unique_covs]")

unique_covs = covmat.astype(bool).astype(int).drop_duplicates()

P = len(pass_tcs)
F = len(fail_tcs)
unique_covs["e_p"] = covmat_bool[pass_tcs].sum(axis=1)
unique_covs["e_f"] = covmat_bool[fail_tcs].sum(axis=1)
unique_covs["n_p"] = P - e_p
unique_covs["n_f"] = F - e_f

unique_covs["Sop2"] = unique_covs.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
unique_covs["Sochiai"] = unique_covs.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
unique_covs.sort_values("Sop2", ascending=False, inplace=True)


display(unique_covs)
pd.reset_option("display.max_columns")


[fault_idx covs]


36  27  9  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1   
                                                             423   1   1   1   

                                                                   5  14  16  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1  1   1    
                                                             423   1  1   1    

                                                                   20  2  21  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1  1    
                                                             423   1   1  1    

                                                                   29  1  24  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1  1    
                                                             423   1   1  1    

                                                                   3  8  25  \
file                                        function         line             
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1  1  1    
                                                             423   1  1  1    

                                                                   30  18  33  \
file                                        function         line               
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1    
                                                             423   1   1   1    

                                                                   28  11  6  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1   
                                                             423   1   1   1   

                                                                   19  4  17  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1  1    
                                                             423   1   1  1    

                                                                   13  23  15  \
file                                        function         line               
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1    
                                                             423   1   1   1    

                                                                   10  22  34  \
file                                        function         line               
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1    
                                                             423   1   1   1    

                                                                   12  7  31  \
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1  1    
                                                             423   1   1  1    

                                                                   35  26  32  
file                                        function         line              
/home/workspace/epan/ftypes/ftype-integer.c uinteger_to_repr 429   1   1   1   
                                                             423   1   1   1

[unique_covs]


,,,36,27,9,5,14,16,20,2,21,29,1,24,3,8,25,30,18,33,28,11,6,19,4,17,13,23,15,10,22,34,12,7,31,35,26,32,e_p,e_f,n_p,n_f,Sop2,Sochiai
file,function,line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
/home/workspace/epan/proto.c,save_same_name_hfinfo,420,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,29,3,4,0,2.147059,0.306186
/home/workspace/epan/dissectors/packet-knxip_decrypt.c,encrypt_block,35,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,31,3,2,0,2.088235,0.297044
/home/workspace/epan/dissectors/packet-xmcp.c,dissect_xmcp_heur,1045,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,32,3,1,0,2.058824,0.292770
/home/workspace/epan/to_str.c,bytes_to_hexstr_punct,166,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,32,3,1,0,2.058824,0.292770
/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,174,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,33,3,0,0,2.029412,0.288675
/home/workspace/epan/ftypes/ftype-integer.c,integer_to_repr,400,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1,13,2,20,1,1.617647,0.298142
/home/workspace/epan/wmem/wmem_tree.c,wmem_tree_lookup32_le,572,1,1,0,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1,14,2,19,1,1.588235,0.288675
/home/workspace/epan/tvbuff.c,tvb_find_tvb,4247,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,0,1,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,16,2,17,1,1.529412,0.272166
/home/workspace/epan/ftypes/ftype-time.c,cmp_ne,40,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,1,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,17,2,16,1,1.500000,0.264906


# 감상

method의 수에 비해, 이 들을 구별할 수 있는 test의 수가 너무 적은 것 같다 (test 수: 36)

faulty method의 ep가 33이고 (max agg) top ranked method의 ep가 29; 그리 큰 차이가 나지 않는다. 그러나 faulty method의 rank는 6707.

max tie breaker의 위에서부터 ranking은 (3, 3, 3, 1061, 1061, 1061, ...) 이다.

그리고 faulty method보다 다른 method들의 크기가 훨씬 커서, voting도 결과가 좋을 수 없다.

마지막으로, hierarchy도 buggy method의 coverage가 모든 test에 의해서 다 cover 되기 때문에, 그 보다 passing test가 적은 spectrum 모두의 descendent인 상황이라 coverage 상으로도 unique한 점이 없다.